# 1.Library Import

In [2]:
import yaml
from box import Box

import torch
import torch.nn as nn
import torch.optim as optim

import simmim
from swin_v2 import SwinTransformerV2

from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
import time

from timm.data import Mixup
import transformers

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2.Configuration

In [15]:
simmim_config = yaml.load(open('config/pretrain.yaml'), Loader=yaml.FullLoader)
simmim_config

{'MODEL': {'TYPE': 'swinv2',
  'NAME': 'simmim_pretrain',
  'DROP_PATH_RATE': 0.0,
  'SWIN': {'EMBED_DIM': 96,
   'DEPTHS': [2, 2, 6, 2],
   'NUM_HEADS': [3, 6, 12, 24],
   'WINDOW_SIZE': 6,
   'PATCH_SIZE': 4}},
 'DATA': {'IMG_SIZE': 192,
  'MASK_PATCH_SIZE': 32,
  'MASK_RATIO': 0.6,
  'BATCH_SIZE': 1024,
  'NUM_WORKERS': 24,
  'DATA_PATH': '../../data/sports'},
 'TRAIN': {'EPOCHS': 100,
  'WARMUP_EPOCHS': 10,
  'BASE_LR': 0.0014,
  'WEIGHT_DECAY': 0.05,
  'CLIP_GRAD': 5}}

In [4]:
encoder_config = {'img_size':simmim_config['DATA']['IMG_SIZE'], 
                'patch_size':simmim_config['MODEL']['SWIN']['PATCH_SIZE'], 
                'in_chans':3, 
                'num_classes':100,
                'embed_dim':simmim_config['MODEL']['SWIN']['EMBED_DIM'], 
                'depths':simmim_config['MODEL']['SWIN']['DEPTHS'], 
                'num_heads':simmim_config['MODEL']['SWIN']['NUM_HEADS'],           
                'window_size':simmim_config['MODEL']['SWIN']['WINDOW_SIZE'], 
                'mlp_ratio':4., 
                'qkv_bias':True, 
                'qk_scale':None,
                'drop_rate':0., 
                'attn_drop_rate':0., 
                'drop_path_rate':simmim_config['MODEL']['DROP_PATH_RATE'],
                'norm_layer':nn.LayerNorm, 
                'patch_norm':True, 
                'pretrained_window_sizes':[0,0,0,0],
                'ape':True}

encoder_stride = 32
in_chans = encoder_config['in_chans']
patch_size = encoder_config['patch_size']

# 3.Load SimMIM

In [4]:
encoder = simmim.SwinTransformerV2ForSimMIM(**encoder_config)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
model = simmim.SimMIM( encoder=encoder, 
                       encoder_stride=encoder_stride, 
                       in_chans=in_chans, 
                       patch_size=patch_size)

## Mask Generator Test

In [6]:
mask_generator = simmim.MaskGenerator(input_size=224,
                                      mask_patch_size=28,
                                      model_patch_size=28,
                                      mask_ratio=.6)
mask = mask_generator()
mask

array([[1, 0, 0, 0, 1, 0, 0, 0],
       [1, 1, 0, 1, 1, 0, 1, 0],
       [1, 1, 1, 0, 0, 0, 1, 1],
       [1, 1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1]])

In [7]:
print(f"생성된 mask의 비율은 {mask.sum() / (mask.shape[0]*mask.shape[1])*100}%")

생성된 mask의 비율은 60.9375%


## SimMIM DataLoader

In [17]:
simmim_config = Box(simmim_config)
dataloader = simmim.build_loader_simmim(simmim_config)

samples = next(iter(dataloader))
len(samples)

3

In [9]:
samples[0].shape, samples[1].shape, samples[2].shape 

(torch.Size([1024, 3, 192, 192]),
 torch.Size([1024, 48, 48]),
 torch.Size([1024]))

## Hyper Parameters and etc.

In [18]:
base_lr = float(simmim_config.TRAIN.BASE_LR)
weight_decay = simmim_config.TRAIN.WEIGHT_DECAY
optimizer = optim.AdamW(model.parameters(), lr=base_lr, weight_decay=weight_decay)
warmup_epochs = simmim_config.TRAIN.WARMUP_EPOCHS
train_epochs = simmim_config.TRAIN.EPOCHS

scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, 
                                                        num_warmup_steps=warmup_epochs*len(dataloader), 
                                                        num_training_steps=train_epochs*len(dataloader),
                                                        num_cycles=0.5)

In [11]:
device = 'cuda:3'
model.to(device)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True


model_save = True
simmim_path = '../../models/swin2/simmim.pth'

# 4.Train SimMIM

In [ ]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(train_epochs):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        image, mask = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            loss = model(image, mask)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        if simmim_config.TRAIN.CLIP_GRAD:
            clip_grad_norm_(model.parameters(), max_norm=simmim_config.TRAIN.CLIP_GRAD)
        else:
            clip_grad_norm_(model.parameters())

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    losses.append(epoch_loss)

    # 모델 저장
    if epoch_loss < best_loss:
        
        best_loss = epoch_loss
        model_save = model_save
        if model_save:
            torch.save(model.state_dict(), simmim_path)
        
    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False    
        
    print(text)


Epoch 1: 100%|██████████| 14/14 [00:19<00:00,  1.41s/it]


	Loss: 1.1431, LR: 0.00014000000000000001, Duration: 20.80 sec - model saved!


Epoch 2: 100%|██████████| 14/14 [00:16<00:00,  1.19s/it]


	Loss: 1.0948, LR: 0.00028000000000000003, Duration: 18.31 sec - model saved!


Epoch 3: 100%|██████████| 14/14 [00:17<00:00,  1.24s/it]


...

	Loss: 0.5173, LR: 1.7051648181230617e-06, Duration: 17.74 sec



Epoch 99: 100%|██████████| 14/14 [00:17<00:00,  1.25s/it]

	Loss: 0.5166, LR: 4.264210866329665e-07, Duration: 18.66 sec



Epoch 100: 100%|██████████| 14/14 [00:17<00:00,  1.24s/it]

	Loss: 0.5177, LR: 0.0, Duration: 18.38 sec


## Del SimMIM Model

In [12]:
model.cpu()
del model
torch.cuda.empty_cache()

# 5.Load Swin V2 for stage-2 training

In [5]:
model = SwinTransformerV2(pretrained_window_sizes=[6,6,6,6], ape=True, drop_path_rate=0.3)
model.state_dict().keys()

/opt/conda/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025831440/work/aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


odict_keys(['absolute_pos_embed', 'embeddings.patch_embeddings.weight', 'embeddings.patch_embeddings.bias', 'embeddings.norm.weight', 'embeddings.norm.bias', 'stages.0.blocks.0.attn_mask', 'stages.0.blocks.0.attn.t_scale', 'stages.0.blocks.0.attn.relative_coords_table', 'stages.0.blocks.0.attn.relative_position_index', 'stages.0.blocks.0.attn.crpb_mlp.0.weight', 'stages.0.blocks.0.attn.crpb_mlp.0.bias', 'stages.0.blocks.0.attn.crpb_mlp.3.weight', 'stages.0.blocks.0.attn.qkv.weight', 'stages.0.blocks.0.attn.qkv.bias', 'stages.0.blocks.0.attn.proj.weight', 'stages.0.blocks.0.attn.proj.bias', 'stages.0.blocks.0.norm1.weight', 'stages.0.blocks.0.norm1.bias', 'stages.0.blocks.0.ls1.gamma', 'stages.0.blocks.0.mlp.fc1.weight', 'stages.0.blocks.0.mlp.fc1.bias', 'stages.0.blocks.0.mlp.fc2.weight', 'stages.0.blocks.0.mlp.fc2.bias', 'stages.0.blocks.0.norm2.weight', 'stages.0.blocks.0.norm2.bias', 'stages.0.blocks.0.ls2.gamma', 'stages.0.blocks.1.attn_mask', 'stages.0.blocks.1.attn.t_scale', 'sta

In [6]:
from torchsummary import summary

summary(model.to('cuda'), (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 56, 56]           4,704
         LayerNorm-2             [-1, 3136, 96]             192
        embeddings-3             [-1, 3136, 96]               0
           Dropout-4             [-1, 3136, 96]               0
            Linear-5              [-1, 49, 288]          27,936
            Linear-6          [-1, 13, 13, 384]           1,152
              ReLU-7          [-1, 13, 13, 384]               0
           Dropout-8          [-1, 13, 13, 384]               0
            Linear-9            [-1, 13, 13, 3]           1,152
          Softmax-10            [-1, 3, 49, 49]               0
          Dropout-11            [-1, 3, 49, 49]               0
           Linear-12               [-1, 49, 96]           9,312
          Dropout-13               [-1, 49, 96]               0
  WindowAttention-14               [-1,

## Default Parameter(weight) Check
- 추후 SimMIM 가중치가 제대로 불러와졌는지 확인용

In [7]:
model.state_dict()['embeddings.patch_embeddings.weight'][0]

tensor([[[ 0.0982, -0.0303,  0.0880,  0.0434],
         [-0.0496,  0.1174,  0.0592, -0.0250],
         [ 0.0544,  0.0834,  0.0161, -0.0264],
         [ 0.0629,  0.1349, -0.0920,  0.0250]],

        [[-0.0895,  0.0073, -0.0723,  0.0701],
         [ 0.0850, -0.0042,  0.0474,  0.1190],
         [ 0.1244,  0.0692,  0.0016, -0.0569],
         [-0.1289,  0.0428,  0.0280,  0.0450]],

        [[-0.0107, -0.0314,  0.0549,  0.0370],
         [ 0.1390, -0.0297,  0.0837,  0.1274],
         [-0.1315, -0.0075,  0.0578, -0.1025],
         [ 0.0094,  0.0660, -0.0369,  0.0334]]], device='cuda:0')

In [8]:
model.state_dict()['stages.3.blocks.1.attn.crpb_mlp.3.weight'][0]

tensor([-0.0009,  0.0250,  0.0339, -0.0226,  0.0136, -0.0180,  0.0068,  0.0204,
         0.0155,  0.0087, -0.0077,  0.0046,  0.0079, -0.0220,  0.0043,  0.0077,
         0.0059, -0.0091,  0.0206, -0.0257, -0.0090,  0.0238,  0.0213, -0.0427,
         0.0148,  0.0222, -0.0188,  0.0085, -0.0206,  0.0116, -0.0026, -0.0230,
        -0.0159, -0.0107,  0.0292,  0.0220, -0.0277, -0.0250, -0.0065, -0.0005,
         0.0024, -0.0199, -0.0143, -0.0021, -0.0192,  0.0025, -0.0045,  0.0061,
        -0.0229, -0.0590, -0.0228, -0.0264,  0.0308,  0.0089, -0.0155,  0.0079,
         0.0059,  0.0226,  0.0012,  0.0221,  0.0146, -0.0172,  0.0231,  0.0056,
         0.0128,  0.0022, -0.0232, -0.0272,  0.0022,  0.0171,  0.0316, -0.0098,
        -0.0179, -0.0017,  0.0075, -0.0058,  0.0010, -0.0021, -0.0014,  0.0274,
         0.0195,  0.0198,  0.0244, -0.0016,  0.0289, -0.0054,  0.0121, -0.0034,
         0.0100,  0.0229,  0.0098, -0.0279,  0.0450,  0.0055,  0.0160, -0.0413,
         0.0014, -0.0169,  0.0116,  0.04

## Load Swin v2 config

In [9]:
swin_config = yaml.load(open('config/train.yaml'), Loader=yaml.FullLoader)
swin_config

{'MODEL': {'TYPE': 'swinv2',
  'NAME': 'simmim_train',
  'PRETRAINED': '../../models/swin2/simmim.pth',
  'DROP_PATH_RATE': 0.2,
  'SWIN': {'EMBED_DIM': 96,
   'DEPTHS': [2, 2, 6, 2],
   'NUM_HEADS': [3, 6, 12, 24],
   'WINDOW_SIZE': 7,
   'PATCH_SIZE': 4}},
 'DATA': {'IMG_SIZE': 224,
  'MASK_PATCH_SIZE': 32,
  'MASK_RATIO': 0.6,
  'BATCH_SIZE': 960,
  'NUM_WORKERS': 24,
  'DATA_PATH': '../../data/sports'},
 'TRAIN': {'EPOCHS': 20,
  'WARMUP_EPOCHS': 10,
  'BASE_LR': '1e-4',
  'WEIGHT_DECAY': 0.05,
  'CLIP_GRAD': 5}}

## Load weight from SimMIM Model
- Different Image/Window Size
- Image와 Window 사이즈의 비율은 맞춰야함
  ex) 192÷6 = 224÷7 = 32

In [10]:
def load_pretrained(config, model):
    print(f"==============> Loading weight {config.MODEL.PRETRAINED} for fine-tuning......")
    state_dict = torch.load(config.MODEL.PRETRAINED, map_location='cpu')

    # remain encoder only
    not_encoder_keys = [k for k in state_dict.keys() if 'encoder' not in k]
    for k in not_encoder_keys:
        del state_dict[k]
        
    # remove prefix encoder.
    state_dict = {k.replace('encoder.', ''):v for k, v in state_dict.items()}

    # delete relative_position_index since we always re-init it
    relative_position_index_keys = [k for k in state_dict.keys() if "relative_position_index" in k]
    for k in relative_position_index_keys:
        del state_dict[k]

    # delete relative_coords_table since we always re-init it
    relative_position_index_keys = [k for k in state_dict.keys() if "relative_coords_table" in k]
    for k in relative_position_index_keys:
        del state_dict[k]

    # delete attn_mask since we always re-init it
    attn_mask_keys = [k for k in state_dict.keys() if "attn_mask" in k]
    for k in attn_mask_keys:
        del state_dict[k]

    # bicubic interpolate relative_position_bias_table if not match
    relative_position_bias_table_keys = [k for k in state_dict.keys() if "relative_position_bias_table" in k]
    for k in relative_position_bias_table_keys:
        relative_position_bias_table_pretrained = state_dict[k]
        relative_position_bias_table_current = model.state_dict()[k]
        L1, nH1 = relative_position_bias_table_pretrained.size()
        L2, nH2 = relative_position_bias_table_current.size()
        if nH1 != nH2:
            print(f"Error in loading {k}, passing......")
        else:
            if L1 != L2:
                # bicubic interpolate relative_position_bias_table if not match
                S1 = int(L1 ** 0.5)
                S2 = int(L2 ** 0.5)
                relative_position_bias_table_pretrained_resized = torch.nn.functional.interpolate(
                    relative_position_bias_table_pretrained.permute(1, 0).view(1, nH1, S1, S1), size=(S2, S2),
                    mode='bicubic')
                state_dict[k] = relative_position_bias_table_pretrained_resized.view(nH2, L2).permute(1, 0)

    # bicubic interpolate absolute_pos_embed if not match
    absolute_pos_embed_keys = [k for k in state_dict.keys() if "absolute_pos_embed" in k]
    for k in absolute_pos_embed_keys:
        # dpe
        absolute_pos_embed_pretrained = state_dict[k]
        absolute_pos_embed_current = model.state_dict()[k.replace('encoder.','')]
        _, L1, C1 = absolute_pos_embed_pretrained.size()
        _, L2, C2 = absolute_pos_embed_current.size()
        if C1 != C1:
            print(f"Error in loading {k}, passing......")
        else:
            if L1 != L2:
                S1 = int(L1 ** 0.5)
                S2 = int(L2 ** 0.5)
                absolute_pos_embed_pretrained = absolute_pos_embed_pretrained.reshape(-1, S1, S1, C1)
                absolute_pos_embed_pretrained = absolute_pos_embed_pretrained.permute(0, 3, 1, 2)
                absolute_pos_embed_pretrained_resized = torch.nn.functional.interpolate(
                    absolute_pos_embed_pretrained, size=(S2, S2), mode='bicubic')
                absolute_pos_embed_pretrained_resized = absolute_pos_embed_pretrained_resized.permute(0, 2, 3, 1)
                absolute_pos_embed_pretrained_resized = absolute_pos_embed_pretrained_resized.flatten(1, 2)
                state_dict[k] = absolute_pos_embed_pretrained_resized

    # check classifier, if not match, then re-init classifier to zero
    head_bias_pretrained = state_dict['classifier.bias']
    Nc1 = head_bias_pretrained.shape[0]
    Nc2 = model.classifier.bias.shape[0]
    if (Nc1 != Nc2):
        torch.nn.init.constant_(model.classifier.bias, 0.)
        torch.nn.init.constant_(model.classifier.weight, 0.)
        del state_dict['classifier.weight']
        del state_dict['classifier.bias']
        print(f"Error in loading classifier head, re-init classifier head to 0")

    msg = model.load_state_dict(state_dict, strict=False)
    print(msg)

    print(f"=> loaded successfully '{config.MODEL.PRETRAINED}'")

    torch.cuda.empty_cache()

In [ ]:
swin_config = Box(swin_config)
load_pretrained(swin_config, model)

## Check Loading Weight Results
- 정상적으로 불러와졌는지 확인

In [12]:
model.state_dict()['embeddings.patch_embeddings.weight'][0]

tensor([[[ 0.0982, -0.0303,  0.0880,  0.0434],
         [-0.0496,  0.1174,  0.0592, -0.0250],
         [ 0.0544,  0.0834,  0.0161, -0.0264],
         [ 0.0629,  0.1349, -0.0920,  0.0250]],

        [[-0.0895,  0.0073, -0.0723,  0.0701],
         [ 0.0850, -0.0042,  0.0474,  0.1190],
         [ 0.1244,  0.0692,  0.0016, -0.0569],
         [-0.1289,  0.0428,  0.0280,  0.0450]],

        [[-0.0107, -0.0314,  0.0549,  0.0370],
         [ 0.1390, -0.0297,  0.0837,  0.1274],
         [-0.1315, -0.0075,  0.0578, -0.1025],
         [ 0.0094,  0.0660, -0.0369,  0.0334]]], device='cuda:0')

In [20]:
model.state_dict()['stages.3.blocks.1.attn.crpb_mlp.3.weight'][0]

tensor([-0.0378, -0.0356, -0.0572,  0.0410,  0.1083,  0.0806, -0.0449,  0.1297,
         0.1483, -0.0354, -0.0157,  0.0207,  0.0078, -0.0074, -0.0733,  0.1325,
         0.0424, -0.0151,  0.0299,  0.0710, -0.0764,  0.0846,  0.0357,  0.0234,
         0.0228, -0.0675,  0.0554,  0.0503,  0.1562,  0.0724, -0.0679, -0.0528,
        -0.0609, -0.0841,  0.0542,  0.0328, -0.0305, -0.0002, -0.0288,  0.1304,
         0.0463,  0.0064, -0.0864, -0.0470, -0.0721, -0.0730, -0.0987, -0.0274,
        -0.0715,  0.0071,  0.0931,  0.0537,  0.1133, -0.0511, -0.0168,  0.0127,
         0.1350,  0.0222, -0.0723,  0.0142, -0.0255, -0.0166, -0.0845,  0.0744,
         0.0075, -0.0324, -0.0903, -0.1243,  0.0328,  0.0239,  0.0102,  0.0438,
        -0.1146, -0.0742, -0.0043,  0.0933,  0.0712, -0.0298, -0.0953,  0.1248,
         0.0784, -0.0411, -0.0242,  0.1539,  0.0641,  0.0427, -0.0101, -0.0378,
        -0.0225,  0.1030,  0.0277, -0.0708, -0.0067,  0.0402,  0.0110,  0.0257,
        -0.0235,  0.0217,  0.0561,  0.01

# 6.Stage-2 Traing
- Supervised pre-training

## Define Transform, Loss, etc.

In [21]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [22]:
# Transforms 정의하기
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8,1), interpolation=transforms.InterpolationMode.LANCZOS),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.9, scale=(0.02, 0.33)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_dir = '../../data/sports'
batch_size = 960

train_path = data_dir+'/train'
valid_path = data_dir+'/valid'
test_path = data_dir+'/test'

# dataset load
train_data = ImageFolder(train_path, transform=train_transform)
valid_data = ImageFolder(valid_path, transform=test_transform)
test_data = ImageFolder(test_path, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [23]:
max_norm = 1.0 # paper : 100 with G variants

model.to(device)
model_path = '../../models/swin2/model_w_simmim.pth'

In [24]:
mixup_fn = Mixup(mixup_alpha=.7, 
                cutmix_alpha=.7, 
                prob=.7, 
                switch_prob=0.5, 
                mode='batch',
                label_smoothing=.1,
                num_classes=100)

epochs = 150

criterion = nn.CrossEntropyLoss(label_smoothing=0.)

## Layer-Wise Learning Rate Decay ★

In [19]:
layer_names = []
for i, (name, params) in enumerate(model.named_parameters()):
    lr = base_lr
    print(f'{i}: {name}')
    layer_names.append(name)

0: absolute_pos_embed
1: embeddings.patch_embeddings.weight
2: embeddings.patch_embeddings.bias
3: embeddings.norm.weight
4: embeddings.norm.bias
5: stages.0.blocks.0.attn.t_scale
6: stages.0.blocks.0.attn.crpb_mlp.0.weight
7: stages.0.blocks.0.attn.crpb_mlp.0.bias
8: stages.0.blocks.0.attn.crpb_mlp.3.weight
9: stages.0.blocks.0.attn.qkv.weight
10: stages.0.blocks.0.attn.qkv.bias
11: stages.0.blocks.0.attn.proj.weight
12: stages.0.blocks.0.attn.proj.bias
13: stages.0.blocks.0.norm1.weight
14: stages.0.blocks.0.norm1.bias
15: stages.0.blocks.0.ls1.gamma
16: stages.0.blocks.0.mlp.fc1.weight
17: stages.0.blocks.0.mlp.fc1.bias
18: stages.0.blocks.0.mlp.fc2.weight
19: stages.0.blocks.0.mlp.fc2.bias
20: stages.0.blocks.0.norm2.weight
21: stages.0.blocks.0.norm2.bias
22: stages.0.blocks.0.ls2.gamma
23: stages.0.blocks.1.attn.t_scale
24: stages.0.blocks.1.attn.crpb_mlp.0.weight
25: stages.0.blocks.1.attn.crpb_mlp.0.bias
26: stages.0.blocks.1.attn.crpb_mlp.3.weight
27: stages.0.blocks.1.attn.qk

In [20]:
layer_names.reverse()
layer_names[:5]

['classifier.bias',
 'classifier.weight',
 'layernorm.bias',
 'layernorm.weight',
 'stages.3.blocks.1.ls2.gamma']

In [23]:
lr      = 1.4e-3      # paper : 1.4e-3
lr_mult = 0.87  # paper : 0.87
weight_decay = 0.01 # paper : 0.1

param_groups = []
prev_group_name = layer_names[0].split('.')[0]

for idx, name in enumerate(layer_names):
    
    cur_group_name = name.split('.')[0] if 'stage' not in name else name.split('.')[3]
    
    if cur_group_name != prev_group_name:
        lr *= lr_mult
    prev_group_name = cur_group_name
    
    print(f"{idx}: {name}'s lr={lr}")
    
    param_groups += [{'params': [ p for n, p in model.named_parameters() if n == name and p.requires_grad],
                      'lr' : lr,
                      'weight_decay': weight_decay}]

0: classifier.bias's lr=0.0014
1: classifier.weight's lr=0.0014
2: layernorm.bias's lr=0.001218
3: layernorm.weight's lr=0.001218
4: stages.3.blocks.1.ls2.gamma's lr=0.0010596599999999998
5: stages.3.blocks.1.norm2.bias's lr=0.0010596599999999998
6: stages.3.blocks.1.norm2.weight's lr=0.0010596599999999998
7: stages.3.blocks.1.mlp.fc2.bias's lr=0.0010596599999999998
8: stages.3.blocks.1.mlp.fc2.weight's lr=0.0010596599999999998
9: stages.3.blocks.1.mlp.fc1.bias's lr=0.0010596599999999998
10: stages.3.blocks.1.mlp.fc1.weight's lr=0.0010596599999999998
11: stages.3.blocks.1.ls1.gamma's lr=0.0010596599999999998
12: stages.3.blocks.1.norm1.bias's lr=0.0010596599999999998
13: stages.3.blocks.1.norm1.weight's lr=0.0010596599999999998
14: stages.3.blocks.1.attn.proj.bias's lr=0.0010596599999999998
15: stages.3.blocks.1.attn.proj.weight's lr=0.0010596599999999998
16: stages.3.blocks.1.attn.qkv.bias's lr=0.0010596599999999998
17: stages.3.blocks.1.attn.qkv.weight's lr=0.0010596599999999998
18: 

In [28]:
# # 모델 레이어의 이름 추출
# layer_names = []
# for i, (name, params) in enumerate(model.named_parameters()):
#     lr = base_lr
#     print(f'{i}: {name}')
#     layer_names.append(name)

# # 뒷 레이어부터 시작하도록 뒤집기    
# layer_names.reverse()

# # 하이퍼 파라미터 정의
# lr      = 1.4e-3      # paper : 1.4e-3
# lr_mult = 0.87  # paper : 0.87
# weight_decay = 0.01 # paper : 0.1

# param_groups = []
# prev_group_name = layer_names[0].split('.')[0] # 그룹명 초기화

# for idx, name in enumerate(layer_names):    
#     cur_group_name = name.split('.')[0]    
#     if cur_group_name != prev_group_name: # 동일한 그룹에 속하면 동일한 학습율
#         lr *= lr_mult
#     prev_group_name = cur_group_name    
    
#     param_groups += [{'params': [ p for n, p in model.named_parameters() if n == name and p.requires_grad],
#                       'lr' : lr,
#                       'weight_decay': weight_decay}]

In [29]:
optimizer = optim.AdamW(param_groups)
warmup_steps = int(len(train_loader)*(epochs)*0.1)
train_steps = len(train_loader)*(epochs)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, 
                                                        num_warmup_steps=warmup_steps, 
                                                        num_training_steps=train_steps,
                                                        num_cycles=0.5)

## Model Train
- 100에포크 먼저 학습하며 결과 확인하고, 이후 10에포크 학습하며 결과 확인

In [30]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(100):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        model_save = True
        if model_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

Epoch 1: 100%|██████████| 15/15 [00:55<00:00,  3.68s/it]


	Loss: 4.639624277750651, Val Loss: 4.52731466293335, LR: 9.333333333333333e-05, Duration: 56.60 sec - model saved!


Epoch 2: 100%|██████████| 15/15 [00:52<00:00,  3.47s/it]


	Loss: 4.480323600769043, Val Loss: 4.236594200134277, LR: 0.00018666666666666666, Duration: 53.27 sec - model saved!


Epoch 3: 100%|██████████| 15/15 [00:51<00:00,  3.43s/it]


...

	Loss: 1.8517841895421345, Val Loss: 0.7762777805328369, LR: 0.00045294860470029185, Duration: 64.89 sec


Epoch 99: 100%|██████████| 15/15 [01:03<00:00,  4.24s/it]


	Loss: 2.2797441403071086, Val Loss: 0.779818058013916, LR: 0.0004377753846088615, Duration: 65.66 sec


Epoch 100: 100%|██████████| 15/15 [00:58<00:00,  3.89s/it]


	Loss: 2.135902206103007, Val Loss: 0.7159838080406189, LR: 0.0004227441637725902, Duration: 59.28 sec
Epoch 당 평균 소요시간 : 35.48초


In [31]:
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [32]:
# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

,Metric,Value
0,Accuracy,0.856000
1,Precision,0.871734
2,Recall,0.856000
3,F1 Score,0.846234


### 110 Epoch Result

In [33]:
for epoch in range(10):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        model_save = True
        if model_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

Epoch 1: 100%|██████████| 15/15 [01:03<00:00,  4.20s/it]


	Loss: 1.8520782709121704, Val Loss: 0.7338253259658813, LR: 0.0004078630818627275, Duration: 64.75 sec


Epoch 2: 100%|██████████| 15/15 [01:03<00:00,  4.24s/it]


	Loss: 1.9222884893417358, Val Loss: 0.6909153461456299, LR: 0.00039314019724764573, Duration: 65.18 sec - model saved!


Epoch 3: 100%|██████████| 15/15 [01:01<00:00,  4.07s/it]


...

Epoch 9: 100%|██████████| 15/15 [00:59<00:00,  3.98s/it]


	Loss: 2.299123199780782, Val Loss: 0.7535581588745117, LR: 0.0002951673646824094, Duration: 60.80 sec


Epoch 10: 100%|██████████| 15/15 [01:03<00:00,  4.25s/it]


	Loss: 1.9305575052897135, Val Loss: 0.7583569884300232, LR: 0.0002819889858080498, Duration: 64.75 sec
Epoch 당 평균 소요시간 : 39.74초


,Metric,Value
0,Accuracy,0.858000
1,Precision,0.871222
2,Recall,0.858000
3,F1 Score,0.849921


### 120 Epoch Result

In [34]:
for epoch in range(10):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        model_save = True
        if model_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

Epoch 1: 100%|██████████| 15/15 [01:06<00:00,  4.41s/it]


	Loss: 1.8062772512435914, Val Loss: 0.6907758712768555, LR: 0.0002690369672720392, Duration: 67.53 sec - model saved!


Epoch 2: 100%|██████████| 15/15 [01:04<00:00,  4.30s/it]


	Loss: 2.082922848065694, Val Loss: 0.7309015393257141, LR: 0.0002563183228209027, Duration: 65.57 sec


Epoch 3: 100%|██████████| 15/15 [01:03<00:00,  4.21s/it]


...

Epoch 9: 100%|██████████| 15/15 [01:01<00:00,  4.10s/it]


	Loss: 1.9349135001500448, Val Loss: 0.7138493657112122, LR: 0.00017438397691295056, Duration: 62.72 sec


Epoch 10: 100%|██████████| 15/15 [01:03<00:00,  4.22s/it]


	Loss: 1.668072803815206, Val Loss: 0.7394352555274963, LR: 0.00016376888981671546, Duration: 64.41 sec
Epoch 당 평균 소요시간 : 44.04초


,Metric,Value
0,Accuracy,0.876000
1,Precision,0.890544
2,Recall,0.876000
3,F1 Score,0.869555


### 130 Epoch Result

In [35]:
for epoch in range(10):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        model_save = True
        if model_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

Epoch 1: 100%|██████████| 15/15 [00:56<00:00,  3.79s/it]


	Loss: 1.9854470094045003, Val Loss: 0.7712658047676086, LR: 0.00015344418132633855, Duration: 57.76 sec


Epoch 2: 100%|██████████| 15/15 [01:05<00:00,  4.35s/it]


	Loss: 2.2352643807729087, Val Loss: 0.761027991771698, LR: 0.00014341544245360648, Duration: 68.45 sec


Epoch 3: 100%|██████████| 15/15 [01:07<00:00,  4.49s/it]


...

	Loss: 2.268953307469686, Val Loss: 0.7554949522018433, LR: 8.19366849987511e-05, Duration: 62.09 sec


Epoch 10: 100%|██████████| 15/15 [01:02<00:00,  4.17s/it]


	Loss: 1.7179184754689534, Val Loss: 0.7154285907745361, LR: 7.445715177361148e-05, Duration: 63.59 sec
Epoch 당 평균 소요시간 : 48.32초


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Metric,Value
0,Accuracy,0.882000
1,Precision,0.892627
2,Recall,0.882000
3,F1 Score,0.874773


### 140 Epoch Result

In [36]:
for epoch in range(10):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        model_save = True
        if model_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

Epoch 1: 100%|██████████| 15/15 [01:03<00:00,  4.26s/it]


	Loss: 1.8992615302403768, Val Loss: 0.7198853492736816, LR: 6.731636103603565e-05, Duration: 64.87 sec


Epoch 2: 100%|██████████| 15/15 [01:01<00:00,  4.13s/it]


	Loss: 1.879345432917277, Val Loss: 0.761381208896637, LR: 6.0518179650179314e-05, Duration: 63.22 sec


Epoch 3: 100%|██████████| 15/15 [01:04<00:00,  4.32s/it]


...

	Loss: 1.7915082613627116, Val Loss: 0.7334794998168945, LR: 2.280934637599534e-05, Duration: 61.69 sec


Epoch 10: 100%|██████████| 15/15 [01:03<00:00,  4.22s/it]


	Loss: 2.0829975446065268, Val Loss: 0.7404515147209167, LR: 1.8868590594123336e-05, Duration: 64.30 sec
Epoch 당 평균 소요시간 : 52.59초


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Metric,Value
0,Accuracy,0.876000
1,Precision,0.886349
2,Recall,0.876000
3,F1 Score,0.867652


### 150 Epoch Result

In [37]:
for epoch in range(10):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        model_save = True
        if model_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

Epoch 1: 100%|██████████| 15/15 [01:04<00:00,  4.28s/it]


	Loss: 1.7545771678288777, Val Loss: 0.7444794774055481, LR: 1.5296679486336016e-05, Duration: 65.38 sec


Epoch 2: 100%|██████████| 15/15 [00:56<00:00,  3.79s/it]


	Loss: 1.7159150203069051, Val Loss: 0.7417320609092712, LR: 1.2095547305510733e-05, Duration: 58.93 sec


Epoch 3: 100%|██████████| 15/15 [01:05<00:00,  4.38s/it]


...

Epoch 9: 100%|██████████| 15/15 [01:03<00:00,  4.25s/it]


	Loss: 2.0819226503372192, Val Loss: 0.7439556121826172, LR: 1.8953117437680023e-07, Duration: 64.65 sec


Epoch 10: 100%|██████████| 15/15 [01:04<00:00,  4.29s/it]


	Loss: 1.7390932083129882, Val Loss: 0.7439295649528503, LR: 0.0, Duration: 65.35 sec
Epoch 당 평균 소요시간 : 56.94초


,Metric,Value
0,Accuracy,0.874000
1,Precision,0.887730
2,Recall,0.874000
3,F1 Score,0.867575
